## Machine leaning project
This project is about wine reviews dataset that was found in Kaggle.

Project team members: Annely Liivas, Velda Lauringson, Eduard Rudi

In [24]:
import pandas as pd

import gensim
from gensim.models import Word2Vec

from statistics import fmean

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [31]:
lemmatizer = WordNetLemmatizer()
word2vec_model = gensim.models.Word2Vec.load('word2vec.model')

In [3]:
data_loc = 'data/'

In [4]:
data1 = pd.read_csv(data_loc + 'winemag-data_first150k.csv', index_col=0)
data2 = pd.read_csv(data_loc + 'winemag-data-130k-v2.csv', index_col=0)

In [5]:
data2_train = data2.iloc[:75000]
test = data2.iloc[75000:]

In [6]:
#Panen andmed kokku
train = pd.concat([data1,data2_train])

In [7]:
# Reset index
train.reset_index(drop=True)
test.reset_index(drop=True)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,US,"Lime, sea shell, rainy cement and orange-rind ...",NaN,88,19.0,California,Santa Barbara County,Central Coast,Matt Kettmann,@mattkettmann,Alma Rosa 2015 Chardonnay (Santa Barbara County),Chardonnay,Alma Rosa
1,US,Aromas of warm peach custard meet with baked y...,Claudia Cuvée Estate,88,35.0,California,Adelaida District,Central Coast,Matt Kettmann,@mattkettmann,Alta Colina 2014 Claudia Cuvée Estate Marsanne...,Marsanne,Alta Colina
2,Italy,Made with 70% Sangiovese and 30% Cabernet Sauv...,Cabreo Il Borgo,88,57.0,Tuscany,Toscana,NaN,Kerin O’Keefe,@kerinokeefe,Ambrogio e Giovanni Folonari 2013 Cabreo Il Bo...,Red Blend,Ambrogio e Giovanni Folonari
3,Italy,"Made entirely with Chardonnay, this opens with...",Cabreo La Pietra,88,37.0,Tuscany,Toscana,NaN,Kerin O’Keefe,@kerinokeefe,Ambrogio e Giovanni Folonari 2014 Cabreo La Pi...,Chardonnay,Ambrogio e Giovanni Folonari
4,Italy,Made with 50% Cabernet Sauvignon and 50% Syrah...,Ardito,88,54.0,Tuscany,Toscana,NaN,Kerin O’Keefe,@kerinokeefe,Baracchi 2012 Ardito Red (Toscana),Red Blend,Baracchi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
54967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
54968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
54969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [8]:
#Vaatan, mis toimub
train.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN


In [9]:
reviews_lower = [x.lower() for x in train.description]

In [10]:
cleaned_reviews = [re.sub('[^a-zA-Z]', ' ', x) for x in reviews_lower]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [11]:
reviews = [nltk.sent_tokenize(x) for x in cleaned_reviews]

In [12]:
reviews = [y for x in reviews for y in x]

In [13]:
words = [nltk.word_tokenize(x) for x in reviews]

In [14]:
stop_words = stopwords.words('english')

In [15]:
for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(w) for w in words[i] if w not in stop_words]

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([' '.join(x) for x in words])

In [18]:
freq = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

c:\Users\rudi\Desktop\Jupyter Notebook\machine_learning\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
fmean(word2vec_model.wv['wine'] * freq['wine'])

-0.010831418770249002

In [32]:
desc_score = []
for desc in words:
    word_score = []
    for w in desc:
        # word_score.append(fmean(word2vec_model.wv[w] * freq[w]))
        try:
            word_score.append(fmean(word2vec_model.wv[w] * freq[w]))
        except:
            continue
    desc_score.append(fmean(word_score))

In [36]:
train['desc_score'] = desc_score

In [37]:
train.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title,desc_score
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN,-0.143920
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN,-0.260135
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN,-0.184423
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN,-0.035441
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN,0.169070
